In [2]:
import pandas as pd
import matplotlib.pyplot as plt

renewables_csv = 'renewable-share-energy.csv'
co2_csv = 'consumption-co2-per-capita.csv'
gdp_file = "world_bank_GDP_per_country.csv"

In [3]:
# functions to turn the CSV files into Pandas dataframes

def vertical_to_horizontal(csv_file):
    # Read the CSV file into a DataFrame
    dataframe = pd.read_csv(csv_file)

    # Pivot the DataFrame based on 'Entity' column
    pivoted_df = dataframe.pivot_table(index='Year', columns='Code', values=dataframe.columns[3]).reset_index()
    
    return pivoted_df

# flipping the data and removing unused years.

renewables_flipped = vertical_to_horizontal(renewables_csv)
renewables_flipped = renewables_flipped[(renewables_flipped['Year'] >= 1990) & (renewables_flipped['Year'] <= 2020)]

co2_flipped = vertical_to_horizontal(co2_csv)
co2_flipped = co2_flipped[(co2_flipped['Year'] >= 1990) & (co2_flipped['Year'] <= 2020)]

non_country_entities = [
    'High-income countries', 'South America', 'Europe', 'Oceania', 'Africa',
    'Lower-middle-income countries', 'Upper-middle-income countries', 'World',
    'North America', 'European Union (27)', 'Asia', 'OWID_WRL'
]

def filter_countries_1990_2020(df):
    # Filter the DataFrame to keep only rows for 1990 and 2020
    df_1990 = df[df['Year'] == 1990]
    df_2020 = df[df['Year'] == 2020]

    # Extract the countries that have data in both 1990 and 2020
    countries_1990 = set(df_1990.columns[df_1990.notnull().any()])
    countries_2020 = set(df_2020.columns[df_2020.notnull().any()])

    countries_both_years = list(countries_1990.intersection(countries_2020))

    # Filter the original DataFrame to include only columns (countries) present in both 1990 and 2020
    filtered_df = df[df.columns[df.columns.isin(countries_both_years)]]

    return filtered_df

renewables_flipped = filter_countries_1990_2020(renewables_flipped)
co2_flipped = filter_countries_1990_2020(co2_flipped)

def filter_common_countries(df1, df2):
    countries_df1 = set(df1.columns)
    countries_df2 = set(df2.columns)
    common_countries = list(countries_df1.intersection(countries_df2))

    # Exclude 'Year' from removal if present in common columns
    if 'Year' in common_countries:
        common_countries.remove('Year')

    # Filter DataFrames to include only common countries (excluding 'Year')
    filtered_df1 = df1[['Year'] + common_countries]
    filtered_df2 = df2[['Year'] + common_countries]

    return filtered_df1, filtered_df2

renewables_filtered, co2_filtered = filter_common_countries(renewables_flipped, co2_flipped)

# Remove non-country entities from the DataFrames
renewables_filtered = renewables_filtered.drop(columns=non_country_entities, errors='ignore')
renewables_filtered.set_index('Year', inplace=True)

co2_filtered = co2_filtered.drop(columns=non_country_entities, errors='ignore')
co2_filtered.set_index('Year', inplace=True)


display(renewables_filtered)
display(co2_filtered)


Code,IRN,LTU,UKR,SVN,JPN,ROU,AUS,IND,PAK,BGD,...,NZL,COL,IDN,USA,TWN,ISR,DNK,DEU,CHE,FRA
Year,,,,,,,,,,,,,,,,,,,,,
1990,2.690250,0.606307,0.973168,12.690258,5.605646,4.590245,4.459392,8.542830,16.537376,3.539271,...,41.426918,32.548810,3.731863,4.724676,3.343195,0.006869,1.266486,1.336941,26.992222,6.222642
1991,1.883583,0.468239,1.156305,15.370844,5.917130,6.907993,4.740063,9.029866,16.701480,3.477609,...,40.412006,31.965420,3.812470,4.724286,1.969026,0.013515,1.458773,1.270958,28.263730,6.266540
1992,2.795918,0.703257,0.921140,15.600477,5.080176,6.283785,4.731717,8.140624,17.344547,3.028885,...,37.466970,24.941273,4.408543,4.261231,3.078827,0.055975,2.040864,1.520455,28.394741,7.399245
1993,3.457789,1.136311,1.471511,13.463441,5.666120,7.054651,4.759429,8.035936,17.234846,2.104329,...,40.207150,28.616268,3.847653,4.571410,1.855712,0.046212,2.361985,1.583544,31.083033,6.968883
1994,2.021462,1.514459,1.890063,14.166087,4.097035,7.641243,4.527182,8.667226,16.347654,2.761680,...,41.244785,30.409860,3.521223,4.254817,2.141311,0.039183,2.132141,1.739633,32.318325,8.563506
1995,2.326306,1.118191,1.591071,12.907960,4.652009,8.897041,4.239725,7.686768,16.378126,1.026995,...,42.026230,29.690346,3.539269,4.762141,1.982554,0.037869,2.313322,1.889190,30.771350,7.823903
1996,2.264600,0.897941,1.480491,13.877275,4.532677,8.448901,4.236591,6.789634,16.762867,1.947564,...,39.501213,31.470236,3.508815,5.018571,1.902318,0.035081,2.187726,1.682394,26.179207,6.882894
1997,1.481070,0.881222,1.752618,11.434757,4.951632,9.899969,4.145840,6.570900,12.898634,1.803428,...,35.658750,27.651566,2.477279,5.115221,1.989283,0.033418,3.380884,1.803517,28.932562,6.803889
1998,1.792134,1.161542,2.900869,12.727215,5.170877,11.665590,4.037756,7.381819,15.707119,2.037124,...,38.804770,26.757204,4.014905,4.695756,2.213548,0.031755,4.675769,1.970565,28.326654,6.470214


Code,IRN,LTU,UKR,SVN,JPN,ROU,AUS,IND,PAK,BGD,...,NZL,COL,IDN,USA,TWN,ISR,DNK,DEU,CHE,FRA
Year,,,,,,,,,,,,,,,,,,,,,
1990,3.627438,11.455255,9.703509,8.292361,10.653232,7.047411,14.263723,0.660851,0.626136,0.170461,...,7.909789,1.900071,0.766681,20.347502,8.498589,10.324067,12.070633,15.016723,12.773527,8.731838
1991,3.681442,12.141571,8.813906,7.840275,10.770959,5.684372,14.118852,0.671166,0.591574,0.177141,...,7.759744,1.816593,0.869577,19.725270,8.428927,10.146182,12.760571,14.304148,12.969457,8.906309
1992,3.695041,13.901977,8.487364,9.917977,11.439899,5.267738,14.113060,0.712204,0.633098,0.185828,...,8.019308,1.982240,0.984626,20.175217,10.479739,11.444240,13.434437,15.013348,13.929676,9.582179
1993,3.573809,8.606816,7.187042,9.200810,11.181535,4.918006,14.031878,0.703520,0.655067,0.185000,...,7.932907,1.995550,1.018359,20.126003,9.875538,11.196589,12.937329,14.365218,12.585456,8.835357
1994,3.912227,7.652970,5.299610,8.747766,11.734203,4.601578,14.365695,0.724505,0.687156,0.200583,...,8.037168,2.056611,1.025336,20.247800,9.843179,11.473627,13.215765,13.985097,12.370958,8.592658
1995,4.114596,7.510003,4.263806,9.452646,11.958007,4.680104,14.520001,0.746073,0.665225,0.223746,...,8.254908,1.844342,1.037032,20.217650,10.064426,11.837488,13.025332,14.024795,12.833714,8.720812
1996,4.290317,6.154727,5.650606,9.156820,11.876456,4.520937,14.703377,0.781769,0.720269,0.226845,...,8.594522,1.809466,1.139294,20.590729,9.266758,11.911931,13.900544,13.792063,15.230000,8.713947
1997,4.267131,5.894873,5.491249,9.444234,11.651935,4.044925,14.954647,0.802062,0.690762,0.238152,...,8.958088,1.918751,1.204452,20.717155,10.079609,12.090898,12.782030,13.213906,13.527470,8.429442
1998,4.856788,6.070131,4.883518,9.560371,10.863544,3.963754,15.831035,0.803076,0.666208,0.232896,...,8.258272,1.894471,0.667905,21.054504,9.908911,12.100624,12.378810,13.229395,16.401410,8.743930


In [10]:
def gdp_dataframe(directory):
    dataframe = pd.read_csv(directory)
    dataframe.set_index('Year', inplace=True)

    transposed_df = dataframe.transpose()
    
    return transposed_df

In [11]:
gdp_filtered = gdp_dataframe(gdp_file)

In [22]:
# Renewables GDP per capita function

common_columns = co2_filtered.columns.intersection(gdp_filtered.columns)

# Creating the DataFrame with NaN values
df_ratio_CO2percapita_perGDP = pd.DataFrame(index=co2_filtered.index, columns=common_columns)

# Performing the division for common columns while handling missing values
for col in common_columns:
    df_ratio_CO2percapita_perGDP[col] = co2_filtered[col].combine_first(gdp_filtered[col]) / gdp_filtered[col]

display(df_ratio_CO2percapita_perGDP)

,IRN,LTU,UKR,SVN,JPN,ROU,AUS,IND,PAK,BGD,...,ITA,NZL,COL,IDN,USA,ISR,DNK,DEU,CHE,FRA
Year,,,,,,,,,,,,,,,,,,,,,
1990,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1991,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1992,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1993,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1994,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1995,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1996,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1997,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1998,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
